In [21]:
import pandas as pd
import numpy as np
import string
import re
import tiktoken
embedding_encoding = "cl100k_base"
max_tokens = 8000
file_name = 'context_03-31-2023.csv'
collected_data = pd.read_csv(file_name)

In [22]:
#collected_data.drop(collected_data[collected_data['Content'].str.contains('Nvidia')])
collected_data.drop(collected_data[(collected_data['Content'].str.contains('Nvidia') == False)].index, inplace=True)
print(collected_data)



ValueError: The truth value of a Int64Index is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [82]:

#removes double lines and single new lines
collected_data["Content"] = collected_data["Content"].replace(to_replace = r'\n\n', value = '', regex=True)
collected_data["Content"] = collected_data["Content"].replace(to_replace = r'\n', value = '', regex=True)
#removes escape charaters
collected_data["Content"] = collected_data["Content"].replace(to_replace = r'\'', value = '', regex=True)

#removes any URL
collected_data["Content"] = collected_data["Content"].replace(to_replace = r"https?://\S+|www.\.\S+", value = '', regex=True)

#removes any html
collected_data["Content"] = collected_data["Content"].replace(to_replace = r"<.*?>", value = '', regex=True)

#removes any emojis
collected_data["Content"] = collected_data["Content"].replace(to_replace = "["u"\U0001F600-\U0001F64F"
                                                                              u"\U0001F300-\U0001F5FF"
                                                                              u"\U0001F680-\U0001F6FF"
                                                                              u"\U0001F1E0-\U0001F1FF"
                                                                              u"\U00002702-\U000027B0"
                                                                              u"\U000024C2-\U0001F251"
                                                                              "]+", value = '', regex=True)




In [83]:
collected_data["Combined"] = (
    "Title: " + collected_data.Title.str.strip() + "; Date: " + collected_data.Date.str.strip()
    + "; Content: " + collected_data.Content.str.strip()
)
encoding = tiktoken.get_encoding(embedding_encoding)

collected_data["n_tokens"] = collected_data.Combined.apply(lambda x: len(encoding.encode(x)))
collected_data = collected_data[collected_data.n_tokens <= max_tokens]




In [89]:
collected_data.loc[collected_data['n_tokens'].idxmin()]


URL         https://www.fool.com/investing/2023/03/20/is-t...
Title                 Is the AI Hype Making Nvidia Too Risky?
Date                                               2023-03-20
Content     Nvidia (NVDA 1.44%) is one of the main winners...
Combined    Title: Is the AI Hype Making Nvidia Too Risky?...
n_tokens                                                  115
Name: 15, dtype: object

In [102]:
from collections import Counter

#Count unique words
def counter_word(text):
    count = Counter()
    for i in text.values:
        for word in i.split():
            count[word] -= 1
    return count

In [103]:
text = collected_data["Content"]
counter = counter_word(text)

In [60]:
len(counter)

934

In [106]:
counter.most_common(50)

[('But,', -1),
 ('topping', -1),
 ('serving', -1),
 ('glass', -1),
 ('half-full', -1),
 ('half-empty.', -1),
 ('reversing', -1),
 ('mid-February', -1),
 ('peak.', -1),
 ('breakdown', -1),
 ('thousand', -1),
 ('words.', -1),
 ('Take', -1),
 ('look:', -1),
 ('bumping', -1),
 ('Only', -1),
 ('Fiscal', -1),
 ("cards'", -1),
 ("quarters'", -1),
 ('matter.', -1),
 ('permanent', -1),
 ('condition.', -1),
 ('lesser-read', -1),
 ('notes', -1),
 ("Kress'", -1),
 ('explain,', -1),
 ('"The', -1),
 ('year-on-year', -1),
 ('decreases', -1),
 ('reflect', -1),
 ('sell-in', -1),
 ('macro-economic', -1),
 ('demand."', -1),
 ('"we', -1),
 ('[China]."', -1),
 ('She', -1),
 ('laptops', -1),
 ('shelves"', -1),
 ('"delivering', -1),
 ('largest-ever', -1),
 ('leap', -1),
 ('efficiency."', -1),
 ('AI-focused', -1),
 ('resisting', -1),
 ('norm.', -1),
 ('her', -1),
 ('agree.', -1),
 ('26.5%', -1),
 ('Eos', -1),
 ('fastest', -1)]

In [49]:
collected_data.to_csv(file_name)